# Supply Chain Strategy FedEx Part 4 Script
#### Jake Heffron

This script imports/utilizes the following data: weight and destination data from 22,376 shipments in the FedEx network, the transportation costs at each weight and zone for the three relevant service types (HD, GR, and E2), and the zone/transit data for each origin DC/store location.

The script generates every possible logistics cost for each shipment, selects the lowest one under the following constraints:
- The number of transit days must be 2 or below
- The GR service is not chosen if a given shipment is below 70 lbs

Then, the script redistributes some shipments based on the following additional constraints (in this order):
- Stores with shipment counts below 500 are eliminated and their shipments are reassigned to the next-lowest-cost option
- Stores with shipment counts above 2000 are identified and shipments are reassigned until the shipment count is 2000 (with the least possible impact on cost)

The script outputs an Excel workbook containing the original 22,376 shipments with their newly generated total logistics cost, origin location, and service type.

In [55]:
import pandas as pd

In [56]:
# loading data
ship_hist = pd.read_excel("Python Part 4 Input.xlsx", sheet_name="Clean Shipping History")

zips = ["90808", "17339", "08330", "21702", "28214", "33304", "38138", "54304", "55105", "57106", "58321", "60525", "75141", "78523", "80640", "82201", "84107", "84602", "89502", "92195", "95824", "97060"]
transit_tables = {}

for zip in zips:
    transit_tables[zip] = pd.read_excel("Transit Tables by Zip.xlsx", sheet_name=zip)
    print("loaded transit table " + zip)

# transit_tables["90808"].head()

loaded transit table 90808
loaded transit table 17339
loaded transit table 08330
loaded transit table 21702
loaded transit table 28214
loaded transit table 33304
loaded transit table 38138
loaded transit table 54304
loaded transit table 55105
loaded transit table 57106
loaded transit table 58321
loaded transit table 60525
loaded transit table 75141
loaded transit table 78523
loaded transit table 80640
loaded transit table 82201
loaded transit table 84107
loaded transit table 84602
loaded transit table 89502
loaded transit table 92195
loaded transit table 95824
loaded transit table 97060


In [57]:
# We don't need all of the columns from the original shipping history file, so we can cut those out
ship_hist = ship_hist[["Destination", "Dest State", "Weight", "Country"]]

In [58]:
# Loading Rate Tables
services = ["HD", "GR", "E2"]

rate_tables = {}

for i in services:
    table = pd.read_excel("Python Part 4 Input.xlsx", sheet_name="{}".format(i))
    rate_tables[i]=table
    
# Some of the rate tables have inconsistent headers, so here we fix that
rate_tables["GR"] = rate_tables["GR"].rename(columns={"Weight (in Lbs )": "Weight (in Lbs)", "Zone-2": "Zone -02", "Zone-3": "Zone -03", "Zone-4": "Zone -04", "Zone-5": "Zone -05", "Zone-6": "Zone -06", "Zone-7": "Zone -07", "Zone-8": "Zone -08"})
rate_tables["HD"] = rate_tables["HD"].rename(columns={"Weight (in Lbs )": "Weight (in Lbs)"})
    
rate_tables["HD"]

,Weight (in Lbs),Zone -02,Zone -03,Zone -04,Zone -05,Zone -06,Zone -07,Zone -08
0,1,8.64,8.64,8.64,8.72,8.99,9.07,9.18
1,2,8.64,8.79,9.29,9.43,9.77,9.86,10.12
2,3,8.64,9.06,9.62,9.84,10.20,10.37,10.88
3,4,8.64,9.22,9.97,10.30,10.58,10.87,11.47
4,5,8.80,9.29,10.25,10.60,10.86,11.21,11.93
...,...,...,...,...,...,...,...,...
65,66,16.22,19.82,21.90,26.92,32.04,36.96,42.98
66,67,16.35,20.00,22.18,26.99,32.27,37.13,43.08
67,68,16.46,20.18,22.46,27.32,32.43,37.32,43.17
68,69,16.58,20.31,22.74,27.66,32.58,37.49,43.29


Looks good! Now that all of the tables are loaded, we need to pull in the operations cost for each store/DC. While this could be pulled from a workbook, I decided to do it manually.

In [59]:
# Create Operations Cost Dictionary
ops_costs = [.9,.95,2.5,3.6,2.5,3.3,3.15,3.2,3.4,3.5,3.3,3,2.9,4,3.1,3.8,3.1,3,3.1,3.9,3.3,3.7]
transit_table_ops = {zips[i]: ops_costs[i] for i in range(len(zips))}
print(transit_table_ops)

{'90808': 0.9, '17339': 0.95, '08330': 2.5, '21702': 3.6, '28214': 2.5, '33304': 3.3, '38138': 3.15, '54304': 3.2, '55105': 3.4, '57106': 3.5, '58321': 3.3, '60525': 3, '75141': 2.9, '78523': 4, '80640': 3.1, '82201': 3.8, '84107': 3.1, '84602': 3, '89502': 3.1, '92195': 3.9, '95824': 3.3, '97060': 3.7}


#### Generating all possible rates

To generate all possible combined logistics costs for each shipment, we first must pull the relevant zone (and transit time, to be used later) for each of the 22 locations for each shipment. Then, we can look up the cost for the shipments using the zone and the associated weight, then add the operations cost.

In [60]:
for transit_table in transit_tables:
    # First must get zones from all origin DC's
    ship_hist_zone = ship_hist.merge(transit_tables[transit_table], how="left", left_on="Destination", right_on="Destination Zip")
    ship_hist_zone = ship_hist_zone.rename(columns={"Zone": "zone_from_{}".format(transit_table)})
    ship_hist_zone = ship_hist_zone.rename(columns={"Transit": "transit_from_{}".format(transit_table)})
    
    # Pulling in the transit times and zones from each origin. If this has already ran in the script, it will be passed
    try:
        ship_hist[["zone_from_{}".format(transit_table)]] = ship_hist_zone[["zone_from_{}".format(transit_table)]]
        ship_hist[["transit_from_{}".format(transit_table)]] = ship_hist_zone[["transit_from_{}".format(transit_table)]]
    except:
        pass
    
    # This is the most exhaustive part of the script where COMBINED logistics rates are retrieved from each of the 3 rate tables for every shipment
    for rate_table in rate_tables:
        values = []
        for index, row in ship_hist.iterrows():
            rate = rate_tables[rate_table][rate_tables[rate_table]["Weight (in Lbs)"]==row["Weight"]].filter(["Zone -{:02d}".format(int(row["zone_from_{}".format(transit_table)]))], axis=1)
            rate += transit_table_ops[transit_table] # this adds the operations cost to the rate depending on the origin DC
            try:
                # conditional that makes it so GR will not be chosen if shipment is below 70 lbs
                if row["Weight"] <= 70 and rate_table == "GR":
                    values.append(9999)
                else:
                    values.append(rate.iloc[0,0])
            except IndexError:                             # Prevents the code from breaking when there is no relevant rate available--this happens in the HD table where there are no rates for shipments above 70 lbs
                values.append(9999)
            if index == 10000:
                print("iterated through {} rows for service {} from origin {}".format(index, rate_table, transit_table))
        ship_hist["{}_{}_rate".format(transit_table,rate_table)] = values
        print("done with {} for origin {}".format(rate_table, transit_table))

        
ship_hist.head(20)

iterated through 10000 rows for service HD from origin 90808
done with HD for origin 90808
iterated through 10000 rows for service GR from origin 90808
done with GR for origin 90808
iterated through 10000 rows for service E2 from origin 90808
done with E2 for origin 90808
iterated through 10000 rows for service HD from origin 17339
done with HD for origin 17339
iterated through 10000 rows for service GR from origin 17339
done with GR for origin 17339
iterated through 10000 rows for service E2 from origin 17339
done with E2 for origin 17339
iterated through 10000 rows for service HD from origin 08330
done with HD for origin 08330
iterated through 10000 rows for service GR from origin 08330
done with GR for origin 08330
iterated through 10000 rows for service E2 from origin 08330
done with E2 for origin 08330
iterated through 10000 rows for service HD from origin 21702
done with HD for origin 21702
iterated through 10000 rows for service GR from origin 21702
done with GR for origin 21702

,Destination,Dest State,Weight,Country,zone_from_90808,transit_from_90808,90808_HD_rate,90808_GR_rate,90808_E2_rate,zone_from_17339,...,zone_from_95824,transit_from_95824,95824_HD_rate,95824_GR_rate,95824_E2_rate,zone_from_97060,transit_from_97060,97060_HD_rate,97060_GR_rate,97060_E2_rate
0,1001,MA,7,US,8,5,13.14,9999.0,29.60,3,...,8,5,15.54,9999.0,32.00,8,5,15.94,9999.0,32.40
1,1002,MA,11,US,8,5,14.62,9999.0,41.52,3,...,8,5,17.02,9999.0,43.92,8,5,17.42,9999.0,44.32
2,1007,MA,6,US,8,5,12.88,9999.0,27.00,3,...,8,5,15.28,9999.0,29.40,8,5,15.68,9999.0,29.80
3,1012,MA,2,US,8,5,11.02,9999.0,16.92,3,...,8,5,13.42,9999.0,19.32,8,5,13.82,9999.0,19.72
4,1020,MA,5,US,8,5,12.83,9999.0,24.69,3,...,8,5,15.23,9999.0,27.09,8,5,15.63,9999.0,27.49
5,1027,MA,6,US,8,5,12.88,9999.0,27.00,3,...,8,5,15.28,9999.0,29.40,8,5,15.68,9999.0,29.80
6,1028,MA,13,US,8,5,16.10,9999.0,46.82,3,...,8,5,18.50,9999.0,49.22,8,5,18.90,9999.0,49.62
7,1030,MA,8,US,8,5,13.60,9999.0,32.49,3,...,8,5,16.00,9999.0,34.89,8,5,16.40,9999.0,35.29
8,1033,MA,2,US,8,5,11.02,9999.0,16.92,3,...,8,5,13.42,9999.0,19.32,8,5,13.82,9999.0,19.72
9,1036,MA,5,US,8,5,12.83,9999.0,24.69,3,...,8,5,15.23,9999.0,27.09,8,5,15.63,9999.0,27.49


Now, we remove rates from consideration that would make a shipment take more than 2 days. Since the lowest cost will ultimately be chosen, we set the GR and HD rates to a very high number ($9,999) if their transit time is above two days. This way, they will not be later chosen as the best option.

In [61]:

for origin in transit_tables:
    for index, row in ship_hist.iterrows():
        if row["transit_from_{}".format(origin)] > 2:
            ship_hist.at[index, "{}_GR_rate".format(origin)] = 9999
            ship_hist.at[index, "{}_HD_rate".format(origin)] = 9999

ship_hist.head(10)

,Destination,Dest State,Weight,Country,zone_from_90808,transit_from_90808,90808_HD_rate,90808_GR_rate,90808_E2_rate,zone_from_17339,...,zone_from_95824,transit_from_95824,95824_HD_rate,95824_GR_rate,95824_E2_rate,zone_from_97060,transit_from_97060,97060_HD_rate,97060_GR_rate,97060_E2_rate
0,1001,MA,7,US,8,5,9999.0,9999.0,29.60,3,...,8,5,9999.0,9999.0,32.00,8,5,9999.0,9999.0,32.40
1,1002,MA,11,US,8,5,9999.0,9999.0,41.52,3,...,8,5,9999.0,9999.0,43.92,8,5,9999.0,9999.0,44.32
2,1007,MA,6,US,8,5,9999.0,9999.0,27.00,3,...,8,5,9999.0,9999.0,29.40,8,5,9999.0,9999.0,29.80
3,1012,MA,2,US,8,5,9999.0,9999.0,16.92,3,...,8,5,9999.0,9999.0,19.32,8,5,9999.0,9999.0,19.72
4,1020,MA,5,US,8,5,9999.0,9999.0,24.69,3,...,8,5,9999.0,9999.0,27.09,8,5,9999.0,9999.0,27.49
5,1027,MA,6,US,8,5,9999.0,9999.0,27.00,3,...,8,5,9999.0,9999.0,29.40,8,5,9999.0,9999.0,29.80
6,1028,MA,13,US,8,5,9999.0,9999.0,46.82,3,...,8,5,9999.0,9999.0,49.22,8,5,9999.0,9999.0,49.62
7,1030,MA,8,US,8,5,9999.0,9999.0,32.49,3,...,8,5,9999.0,9999.0,34.89,8,5,9999.0,9999.0,35.29
8,1033,MA,2,US,8,5,9999.0,9999.0,16.92,3,...,8,5,9999.0,9999.0,19.32,8,5,9999.0,9999.0,19.72
9,1036,MA,5,US,8,5,9999.0,9999.0,24.69,3,...,8,5,9999.0,9999.0,27.09,8,5,9999.0,9999.0,27.49


In [62]:
columns = []
for col in ship_hist.columns:
    if col[-4:] == "rate":
        columns.append(col)
# print(columns)

# New DataFrame that only contains the possible rates for each shipment

ship_hist_rates = ship_hist.filter(columns)

#### Identifying lowest rate and adjusting to constraints

First, we will choose the lowest rate for each shipment and record the associated store/DC and service. Then, the script evaluates the shipment count at each store, increases the minimum shipment count to keep the store in consideration (called the "limit") by 1. Once the minimum surpasses the current shipment count at a store, that store is removed and the shipments are reassigned to the next-cheapest option via a recalculation of the best rate for every shipment. This process repeats until the minimum store count reaches 500.  While this part of the script is exhaustive, is the only way to ensure each shipment is reassigned to its most cost-effective origin location.

In [63]:
# Determine which rate is the lowest and identify the associated service type and origin

def get_best_rates():
    ship_hist_best = pd.DataFrame({"best_rate": ship_hist_rates.min(axis=1)})
#     print(ship_hist_best.shape)
#     print(ship_hist_rates.shape)

    pairs = []
    for index, row in ship_hist_rates.iterrows():
        for col in ship_hist_rates.columns:
            if row[col] == ship_hist_best.iloc[index,0]:
                pairs.append(col)
                break

#     print(len(pairs))

    ship_hist_best["origin_service_pair"] = pairs
    
    # Creating columns for the new origins and services
    location = ship_hist_best["origin_service_pair"].str.slice(0,5)
    service = ship_hist_best["origin_service_pair"].str.slice(6,8)
    # print(location)
    ship_hist_best["new_origin_location"] = location
    ship_hist_best["new_service_type"] = service
    return ship_hist_best

ship_hist_best = get_best_rates()

# Now stores will be eliminated that have under a certain limit of shipments, with that limit being increased up to the 500-shipment constraint

limit = 1
while limit <= 500:
    for col in ship_hist_rates.columns:
        if len(ship_hist_best["new_origin_location"][ship_hist_best["new_origin_location"]==col[0:5]]) < limit:
            ship_hist_rates.drop(columns=col, inplace=True)
            print("dropped column {}".format(col))
#         else:
#             print("didnt drop column {}".format(col))
            ship_hist_best = get_best_rates()
    print("completed limit {}".format(limit))
    limit += 1
    
# ship_hist_rates.sample(10)
# ship_hist_best.sample(10)


dropped column 21702_HD_rate
dropped column 21702_GR_rate
dropped column 21702_E2_rate
dropped column 84107_HD_rate
dropped column 84107_GR_rate
dropped column 84107_E2_rate
dropped column 92195_HD_rate
dropped column 92195_GR_rate
dropped column 92195_E2_rate
completed limit 1
completed limit 2
completed limit 3
completed limit 4
completed limit 5
dropped column 95824_HD_rate
dropped column 95824_GR_rate
dropped column 95824_E2_rate
completed limit 6
completed limit 7
completed limit 8
completed limit 9
completed limit 10
completed limit 11
completed limit 12
completed limit 13
completed limit 14
completed limit 15
completed limit 16
completed limit 17
completed limit 18
completed limit 19
dropped column 80640_HD_rate
dropped column 80640_GR_rate
dropped column 80640_E2_rate
completed limit 20
completed limit 21
completed limit 22
completed limit 23
completed limit 24
completed limit 25
completed limit 26
completed limit 27
dropped column 78523_HD_rate
dropped column 78523_GR_rate
dro

In [64]:
# ship_hist_rates.sample(10)
ship_hist_rates.columns

Index(['90808_HD_rate', '90808_GR_rate', '90808_E2_rate', '17339_HD_rate',
       '17339_GR_rate', '17339_E2_rate', '28214_HD_rate', '28214_GR_rate',
       '28214_E2_rate', '55105_HD_rate', '55105_GR_rate', '55105_E2_rate',
       '60525_HD_rate', '60525_GR_rate', '60525_E2_rate', '75141_HD_rate',
       '75141_GR_rate', '75141_E2_rate', '84602_HD_rate', '84602_GR_rate',
       '84602_E2_rate', '97060_HD_rate', '97060_GR_rate', '97060_E2_rate'],
      dtype='object')

At this point in the script, there is one store--zip code 75141--with 2096 shipments despite the provided constraint of 2000 shipments to a given store. Luckily, the script can automatically identify which 96 shipments would be least impacted (cost-wise) by reassignment!

First, for all shipments, we identify what the best option would be if store 75141 were to be completely removed from consideration.

In [65]:
# Now we deal with 75141 where there are 2096 shipments (maximum for stores is 2000)
# First we identify the next best option for 75141 shipments
ship_hist_rates_no_75141 = ship_hist_rates.drop(columns=["75141_HD_rate", "75141_GR_rate", "75141_E2_rate"])


ship_hist_best_no_75141 = pd.DataFrame({"no_75141_best_rate": ship_hist_rates_no_75141.min(axis=1)})
#     print(ship_hist_best.shape)
#     print(ship_hist_rates.shape)

pairs_no_75141 = []
for index, row in ship_hist_rates_no_75141.iterrows():
    for col in ship_hist_rates_no_75141.columns:
        if row[col] == ship_hist_best_no_75141.iloc[index,0]:
            pairs_no_75141.append(col)
#             row["second_best_rate"] = row.drop(columns=col).min()
            break

#     print(len(pairs))

ship_hist_best_no_75141["no_75141_origin_service_pair"] = pairs_no_75141
#     print(origin_service_pair)

# Creating columns for the new origins and services
location = ship_hist_best_no_75141["no_75141_origin_service_pair"].str.slice(0,5)
service = ship_hist_best_no_75141["no_75141_origin_service_pair"].str.slice(6,8)
# print(location)
ship_hist_best_no_75141["no_75141_new_origin_location"] = location
ship_hist_best_no_75141["no_75141_new_service_type"] = service

if "no_75141_best_rate" not in ship_hist_best:
    ship_hist_best = pd.concat([ship_hist_best,ship_hist_best_no_75141], axis=1)

ship_hist_best[ship_hist_best["new_origin_location"]=="75141"].sample(10)

,best_rate,origin_service_pair,new_origin_location,new_service_type,no_75141_best_rate,no_75141_origin_service_pair,no_75141_new_origin_location,no_75141_new_service_type
15923,11.69,75141_HD_rate,75141,HD,12.16,84602_E2_rate,84602,E2
16593,15.59,75141_HD_rate,75141,HD,18.39,60525_HD_rate,60525,HD
16793,11.69,75141_HD_rate,75141,HD,12.43,84602_HD_rate,84602,HD
15961,17.50,75141_HD_rate,75141,HD,23.15,84602_HD_rate,84602,HD
21248,13.70,75141_HD_rate,75141,HD,13.80,60525_HD_rate,60525,HD
16310,11.45,75141_E2_rate,75141,E2,12.43,60525_HD_rate,60525,HD
10068,14.21,75141_HD_rate,75141,HD,14.65,60525_HD_rate,60525,HD
10055,38.54,75141_GR_rate,75141,GR,40.58,60525_GR_rate,60525,GR
9797,13.70,75141_HD_rate,75141,HD,13.84,28214_HD_rate,28214,HD
16189,11.70,75141_HD_rate,75141,HD,13.60,60525_HD_rate,60525,HD


Now, we identify the 96 shipments with the most negligible difference in cost between 75141 and the next best option--these will be the rerouted shipments.

In [66]:

ship_hist_best["75141_alt_diff"] = ship_hist_best["no_75141_best_rate"] - ship_hist_best["best_rate"]

# list(ship_hist_best["75141_alt_diff"])
differences = pd.DataFrame({"difference": list(ship_hist_best["75141_alt_diff"]), "index": ship_hist_best.index})
differences = differences.sort_values(by="difference")
# print(differences)


differences_filtered = differences[differences["difference"]>0]

differences_filtered = differences_filtered[0:96]
display(differences_filtered)

,difference,index
19680,0.02,19680
19682,0.02,19682
10015,0.02,10015
9907,0.02,9907
16131,0.02,16131
...,...,...
17813,0.04,17813
9690,0.05,9690
9660,0.05,9660
15748,0.05,15748


It looks like the 96 shipments have a very small (between 2 and 5 cent) difference between using 75141 and the next best option. Awesome! Now, for those 96 shipments, we reassign to the next best option.

In [67]:
# Finally, we reassign the appropriate 96 indexes to their secondary location!
for index, row in ship_hist_best.iterrows():
    if index in list(differences_filtered["index"]):
        ship_hist_best.at[index, "best_rate"] = row["no_75141_best_rate"]
        ship_hist_best.at[index, "origin_service_pair"] = row["no_75141_origin_service_pair"]
        ship_hist_best.at[index, "new_origin_location"] = row["no_75141_new_origin_location"]
        ship_hist_best.at[index, "new_service_type"] = row["no_75141_new_service_type"]
        print("replaced index {}".format(index))

# Dropping columns we don't need in the output
ship_hist_best.drop(columns=["no_75141_best_rate", "no_75141_origin_service_pair", "no_75141_new_origin_location", "no_75141_new_service_type", "75141_alt_diff"], inplace=True)

replaced index 7509
replaced index 8457
replaced index 9352
replaced index 9353
replaced index 9369
replaced index 9444
replaced index 9467
replaced index 9561
replaced index 9565
replaced index 9573
replaced index 9590
replaced index 9645
replaced index 9660
replaced index 9664
replaced index 9690
replaced index 9739
replaced index 9740
replaced index 9746
replaced index 9824
replaced index 9907
replaced index 9909
replaced index 9911
replaced index 9912
replaced index 9928
replaced index 9990
replaced index 9992
replaced index 9999
replaced index 10000
replaced index 10001
replaced index 10007
replaced index 10008
replaced index 10009
replaced index 10010
replaced index 10015
replaced index 10016
replaced index 10642
replaced index 10650
replaced index 10655
replaced index 10659
replaced index 15683
replaced index 15700
replaced index 15724
replaced index 15726
replaced index 15743
replaced index 15744
replaced index 15748
replaced index 16129
replaced index 16131
replaced index 1703

We can quickly calculate the transit days for each shipment with the new origin locations (rather than writing 22 "IF" statements in Excel).

In [71]:
# for index, row in ship_hist.iterrows():
#     for best_index, best_row in ship_hist_best.iterrows():
#         if best_row["new_service_type"] == "E2":
#             ship_hist_best.at[best_index, "transit_time"] = 2
#         else:
#             table = transit_tables[best_row["new_origin_location"]]
#             table.head()
#             ship_hist_best.at[best_index, "transit_time"] = table[table["Destination Zip"]==row["Destination"]]["Transit"]

for index, row in ship_hist_best.iterrows():
        if row["new_service_type"] == "E2":
            ship_hist_best.at[index, "transit_time"] = 2
        else:
            ship_hist_best.at[index,"transit_time"] = ship_hist.loc[index,"transit_from_{}".format(row["new_origin_location"])]

#### Output
Now we're done! Let's write our results to an Excel table and go get a glass of water.

In [72]:
# Write new data to Excel
ship_hist_best.to_excel("Python Part 4 Output.xlsx", sheet_name="Optimized Shipments")

In [73]:
# Original shipping history file with every possible rate for each shipment. Not necessary for deliverable but can help explain script
ship_hist.to_excel("Python 4 Shipments Output.xlsx")